# Modelo

YOLO es una familia de modelos de detección de objetos a escala compuesta entrenados en COCO dataset, e incluye una funcionalidad simple para Test Time Augmentation (TTA), model ensembling, hyperparameter evolution, and export to ONNX, CoreML and TFLite.


## Gradio Inferencia

![](https://i.ibb.co/982NS6m/header.png)

  Este Notebook se acelera opcionalmente con un entorno de ejecución de GPU


----------------------------------------------------------------------

 YOLOv5 Gradio demo

*Author: Ultralytics LLC and Gradio*

# Código

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 7.1 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 9.9 MB 53.4 MB/s 
     |████████████████████████████████| 140 kB 61.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


In [ ]:
pip show boto3

Name: boto3
Version: 1.26.16
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: botocore, jmespath, s3transfer
Required-by: 


In [ ]:
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt gradio # install dependencies

     |████████████████████████████████| 11.6 MB 7.6 MB/s 
     |████████████████████████████████| 182 kB 37.1 MB/s 
     |████████████████████████████████| 127 kB 49.8 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 213 kB 25.5 MB/s 
     |████████████████████████████████| 278 kB 40.1 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 84 kB 2.4 MB/s 
     |████████████████████████████████| 106 kB 43.4 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 2.3 MB 53.0 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 1.6 MB 38.8 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 80 kB 10.1 MB/s 
     |████████████████████████████████| 68 kB 8.1 MB/s 
     |█████████████████████████

In [ ]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Ingresar el codigo que esta en Aplicativo_OceanApp --> TRAIN_GOOGLE_COLAB --> awscli

In [ ]:
import gradio as gr
import pandas as pd
import torch
import logging
import json
import re
import os #Para las variables de entorno
#-------Para conectar con aws--------
import boto3
from botocore.exceptions import NoCredentialsError
import tempfile
import io
#-------------Fin-------------------
from PIL import Image

In [ ]:
!export AWS_CONFIG_FILE=/content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_CONFIG_FILE'] = path
print(os.environ['AWS_CONFIG_FILE'])

/content/drive/My Drive/config/awscli.ini


In [ ]:
def removeStr(string):
    return string.replace(" ", "")

def listJSON(a,b,c,d,e,f,resImg):
      x = re.findall("obo Mar", d)
      y = re.findall("elica", d)
      z = re.findall("elica", f)
      if x:
        d = 'Lobo marino'
      if y:
        d = 'Pelicano'
      if z:
        f = 'Pelicano'
      if(d=='Lobo marino' or d=='Pelicano'):
        if d =='Pelicano\nSp' or d =='Pelicano\nS':
          d = 'Pelicano'
        if f!='Pelicano':
          strlista = '"detail":[{"quantity":"'+str(removeStr(c))+'","description":"'+str(d)+'"}]'
        else:
          strlista = '"detail":[{"quantity":"'+str(removeStr(c))+'","description":"'+str(d)+'"},{"quantity":"'+str(removeStr(e))+'","description":"'+str(f)+'"}]'
        strlist = '{"image":"'+str(removeStr(a))+'","size":"'+str(removeStr(b))+'",'+strlista+','+resImg+'}'
        json_string = json.loads(strlist)
        return json_string

def arrayLista(a,b,c,d):
      x = re.findall("obo Mar", b)
      y = re.findall("elica", b)
      z = re.findall("elica", d)
      if x:
        b = 'Lobo marino'
      if y:
        b = 'Pelicano'
      if z:
        d = 'Pelicano'
      if(b=='Lobo marino' or b=='Pelicano'):
        strlist =[]
        strlist2 =[]
        strlist.append(removeStr(a))
        strlist.append(b)
        if d=='Pelicano':
          strlist2.append(removeStr(c))
          strlist2.append(d)
        strlista = [strlist,strlist2]
        df = pd.DataFrame(strlista,columns=['Cantidad','Especie'])
        return df

#Imagen temporal guardada en upload_file
def tempFileJSON(img_file):
  temp = tempfile.NamedTemporaryFile(mode="wb")
  with temp as jpg:
    jpg.write(img_file)
    print(jpg.name)
    uf = upload_file(jpg.name)
    #uf='"url_details":[{"statusCode":200, "s3_url":"+s3_url+"}]'
    return uf
  print('Exists after close:', os.path.exists(temp.name))

# Envio de imagenes a S3
def upload_file(file_name, bucket=None, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: Json if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name+".jpg")
    if bucket is None:
        bucket = 'oceanapp'
    aws_region = boto3.session.Session().region_name
    # Upload the file
    s3_client = boto3.client('s3')
    try:
        with open(file_name, "rb") as f:
          response = s3_client.upload_fileobj(f, bucket, object_name)
          s3_url = f"https://{bucket}.s3.{aws_region}.amazonaws.com/{object_name}"
          stado = '"url_details":[{"statusCode":200, "s3_url":"'+s3_url+'"}]'
        print(s3_url)
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError as e:
        logging.error(e)
        return False
    return stado

In [ ]:
#------------ Interface-------------
# Images
torch.hub.download_url_to_file('https://i.pinimg.com/564x/18/0b/00/180b00e454362ff5caabe87d9a763a6f.jpg', 'ejemplo1.jpg')
torch.hub.download_url_to_file('https://i.pinimg.com/564x/3b/2f/d4/3b2fd4b6881b64429f208c5f32e5e4be.jpg', 'ejemplo2.jpg')

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True, autoshape=True)  # local model  o google colab

def yolo(size, iou, conf, im):
    try:
      '''Wrapper fn for gradio'''
      # gain
      g = (int(size) / max(im.size))
      # resize
      im = im.resize((int(x * g) for x in im.size), Image.ANTIALIAS)

      model.iou = iou

      model.conf = conf
      # inference
      results2 = model(im)
      # updates results.imgs with boxes and labels
      results2.render()
      results3 = str(results2)
      print(results3)
      #Transformando la img en bytes
      pil_im = Image.fromarray(results2.ims[0])
      b = io.BytesIO()
      pil_im.save(b, 'jpeg')
      im_bytes = b.getvalue()
      fileImg = tempFileJSON(im_bytes)
      lista = listJSON(results3[0:9], results3[11:18] ,results3[19:21],results3[22:32], results3[34:36], results3[36:45], fileImg)
      #print(lista)
      lista2 = arrayLista(results3[19:21],results3[22:32], results3[34:36], results3[37:45])
      return Image.fromarray(results2.ims[0]), lista2, lista
    except Exception as e:
      logging.error(e, exc_info=True)

#-----------
in1 = gr.inputs.Radio(['640', '1280'], label="Tamaño de la imagen", default='640', type='value')
in2 = gr.inputs.Slider(minimum=0, maximum=1, step=0.05, default=0.45, label='NMS IoU threshold')
in3 = gr.inputs.Slider(minimum=0, maximum=1, step=0.05, default=0.50, label='Umbral o  threshold')
in4 = gr.inputs.Image(type='pil', label="Original Image")

out2 = gr.outputs.Image(type="pil", label="Identificación Yolov5")
#out3 = gr.outputs.Dataframe(label="Descripción", headers=['Cantidad','Especie'])
out4 = gr.outputs.JSON(label="JSON")

#-------------- Text-----
title = 'OceanApp'
description = """
<p>
<center>
Sistema para el reconocimiento de las especies en la pesca acompañante de cerco, utilizando redes neuronales convolucionales para una empresa del sector pesquero en los puertos de callao y paracas.
<img src="https://i.pinimg.com/564x/3e/b8/f7/3eb8f7c348dffd7b3dffcafe81fbf2a6.jpg" alt="logo" width="250"/>
</center>
</p>
"""
article = "<p style='text-align: center'><a href='' target='_blank'>Para mas info, clik para ir al white paper</a></p><p style='text-align: center'><a href='https://colab.research.google.com/drive/1j0T8gdLIa0X8fzkIgFpXDoU27BF49RUz?usp=sharing' target='_blank'>Google Colab Demo</a></p><p style='text-align: center'><a href='https://github.com/MssLune/OceanApp-Model' target='_blank'>Repo Github</a></p></center></p>"

examples = [['640',0.45, 0.75,'ejemplo1.jpg'], ['640',0.45, 0.75,'ejemplo2.jpg']]

iface = gr.Interface(yolo, inputs=[in1, in2, in3, in4], outputs=[out2,out4], title=title, description=description, article=article, examples=examples,theme="huggingface", analytics_enabled=False).launch(
    debug=True, share=True)
#Se agrego el share True, antes no lo pedian
iface.launch()

  0%|          | 0.00/36.7k [00:00<?, ?B/s]

  0%|          | 0.00/26.7k [00:00<?, ?B/s]

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-11-25 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cbe95b50b7d44707.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


image 1/1: 426x640 1 Pelicano
Speed: 17.3ms pre-process, 245.3ms inference, 15.5ms NMS per image at shape (1, 3, 448, 640)
/tmp/tmp8lq1b0eh


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/protocols/http/h11_impl.py", line 408, in run_asgi
    self.scope, self.receive, self.send
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/fastapi/applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/co

https://oceanapp.s3.us-east-1.amazonaws.com/tmp8lq1b0eh.jpg
Keyboard interruption in main thread... closing server.


AttributeError: ignored

For YOLOv5 PyTorch Hub inference with **PIL**, **OpenCV**, **Numpy** or **PyTorch** inputs please see the full [YOLOv5 PyTorch Hub Tutorial](https://github.com/ultralytics/yolov5/issues/36).


## Citation

[![DOI](https://zenodo.org/badge/264818686.svg)](https://zenodo.org/badge/latestdoi/264818686)